## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [23]:
# 1. Import the WeatherPy_database.csv file.

city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,0,Mayo,38.8876,-76.5119,38.55,39,0,7.00,US,2022-02-06 18:43:01,clear sky
1,1,Torbay,47.6666,-52.7314,16.75,81,100,17.27,CA,2022-02-06 18:44:52,light snow
2,2,Kavieng,-2.5744,150.7967,81.16,77,82,3.47,PG,2022-02-06 18:44:52,broken clouds
3,3,Srednekolymsk,67.4500,153.6833,-36.83,100,58,1.28,RU,2022-02-06 18:44:53,broken clouds
4,4,Thinadhoo,0.5333,72.9333,82.62,69,75,11.05,MV,2022-02-06 18:44:53,broken clouds


In [51]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 60


In [52]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
10,10,Busselton,-33.6500,115.3333,54.09,53,64,17.92,AU,2022-02-06 18:44:55,broken clouds
27,27,Dunedin,-45.8742,170.5036,52.81,79,0,1.01,NZ,2022-02-06 18:44:59,clear sky
38,38,Hobart,-42.8794,147.3294,53.06,87,5,9.22,AU,2022-02-06 18:45:02,clear sky
56,56,Portland,45.5234,-122.6762,54.28,81,100,3.00,US,2022-02-06 18:45:10,fog
70,70,New Norfolk,-42.7826,147.0587,52.72,74,4,1.95,AU,2022-02-06 18:47:30,clear sky
71,71,Iberia,29.9502,-91.7507,54.97,37,0,4.61,US,2022-02-06 18:48:57,clear sky
72,72,Gat,31.6100,34.7642,54.91,83,0,5.59,IL,2022-02-06 18:48:58,clear sky
78,78,Chalmette,29.9427,-89.9634,57.09,41,0,1.99,US,2022-02-06 18:48:59,clear sky
80,80,Lakes Entrance,-37.8811,147.9810,52.57,91,8,5.59,AU,2022-02-06 18:49:00,clear sky
82,82,Tezu,27.9167,96.1667,52.00,88,100,3.83,IN,2022-02-06 18:49:01,overcast clouds


In [53]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                56
City                   56
Lat                    56
Lng                    56
Max Temp               56
Humidity               56
Cloudiness             56
Wind Speed             56
Country                56
Date                   56
Current Description    56
dtype: int64

In [54]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                56
City                   56
Lat                    56
Lng                    56
Max Temp               56
Humidity               56
Cloudiness             56
Wind Speed             56
Country                56
Date                   56
Current Description    56
dtype: int64

In [55]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Busselton,AU,54.09,broken clouds,-33.6500,115.3333,
27,Dunedin,NZ,52.81,clear sky,-45.8742,170.5036,
38,Hobart,AU,53.06,clear sky,-42.8794,147.3294,
56,Portland,US,54.28,fog,45.5234,-122.6762,
70,New Norfolk,AU,52.72,clear sky,-42.7826,147.0587,
71,Iberia,US,54.97,clear sky,29.9502,-91.7507,
72,Gat,IL,54.91,clear sky,31.6100,34.7642,
78,Chalmette,US,57.09,clear sky,29.9427,-89.9634,
80,Lakes Entrance,AU,52.57,clear sky,-37.8811,147.9810,
82,Tezu,IN,52.00,overcast clouds,27.9167,96.1667,


In [57]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.


In [61]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
10,Busselton,AU,54.09,broken clouds,-33.6500,115.3333,Observatory Guest House
27,Dunedin,NZ,52.81,clear sky,-45.8742,170.5036,Scenic Hotel Southern Cross
38,Hobart,AU,53.06,clear sky,-42.8794,147.3294,Mantra on Collins Hobart
56,Portland,US,54.28,fog,45.5234,-122.6762,Embassy Suites by Hilton Portland Downtown
70,New Norfolk,AU,52.72,clear sky,-42.7826,147.0587,The Shingles Riverside Cottages
71,Iberia,US,54.97,clear sky,29.9502,-91.7507,back to back rv park
72,Gat,IL,54.91,clear sky,31.6100,34.7642,OlusHome
78,Chalmette,US,57.09,clear sky,29.9427,-89.9634,Marina Inn & Suites Chalmette-New Orleans
80,Lakes Entrance,AU,52.57,clear sky,-37.8811,147.9810,The Esplanade Resort & Spa
82,Tezu,IN,52.00,overcast clouds,27.9167,96.1667,Circuit House


In [62]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [64]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [68]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))